In [1]:
from lib.data import raw

import pandas as pd
import numpy as np

import requests


In [2]:
import xmltodict
import datetime

def read_xml_from_url(xml_url: str) -> pd.DataFrame:

    xml_content = requests.get(xml_url).content

    xml_data = xmltodict.parse(xml_content)

    columns = ['id', 'intensity', 'occupation', 'congestion', 'datetime']

    datet = datetime.datetime.strptime(xml_data['pms']['fecha_hora'], '%d/%m/%Y %H:%M:%S')

    data = [[int(x['idelem']), 
                int(x['intensidad']), 
                    int(x['ocupacion']), 
                        int(x['carga']),
                            datet] for x in xml_data['pms']['pm'] if x['error'] == 'N']

    return pd.DataFrame(data, columns=columns)

xml_data = read_xml_from_url('https://datos.madrid.es/egob/catalogo/202087-0-trafico-intensidad.xml')

xml_data

,id,intensity,occupation,congestion,datetime
0,3409,212,2,8,2022-11-01 13:45:05
1,4739,466,7,36,2022-11-01 13:45:05
2,4740,180,4,16,2022-11-01 13:45:05
3,4741,45,0,3,2022-11-01 13:45:05
4,4742,225,1,8,2022-11-01 13:45:05
...,...,...,...,...,...
4133,3817,480,4,35,2022-11-01 13:45:05
4134,10660,600,5,37,2022-11-01 13:45:05
4135,10662,480,3,29,2022-11-01 13:45:05
4136,10661,1560,15,89,2022-11-01 13:45:05


In [3]:
traffic_stations = next(raw.load_dataset(raw.TRAFFIC_STATIONS, 'csv', 2022, verbose=True))

traffic_stations = traffic_stations[['id', 'longitud', 'latitud']][traffic_stations['tipo_elem'] == 'URB']

  Found 48 files
    Dataframe size: (4663, 9)


In [4]:
traffic_stations

,id,longitud,latitud
0,3840,-3.688323,40.430502
1,3841,-3.687256,40.430524
2,3842,-3.691727,40.422132
3,3843,-3.691929,40.421433
4,3844,-3.688470,40.433782
...,...,...,...
4362,3577,-3.775800,40.399330
4363,5167,-3.775626,40.399044
4364,5164,-3.774217,40.396418
4365,5177,-3.772306,40.394035


In [7]:
['week', 'intensity', 'occupation', 'congestion', 'is_holiday', 'daypart']

['week', 'intensity', 'occupation', 'congestion', 'is_holiday', 'daypart']

In [5]:
from lib.data import clean
import numpy as np

prepared_data = xml_data.merge(traffic_stations, on='id', how='left').drop(columns=['id'])

prepared_data['week'] = prepared_data['datetime'].dt.isocalendar().week

hours = {
    # diurno
    0: [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18],
    # nocturno
    1: [23, 0, 1, 2, 3, 4, 5, 6],
    # vespertino
    2: [19, 20, 21, 22]
}

# 0: D (diurno), 1: N (nocturno), 2: E (vespertino)
set_daypart = lambda x: next(dayp for dayp in hours if x in hours[dayp])

prepared_data['year'] = prepared_data['datetime'].dt.year
prepared_data['month'] = prepared_data['datetime'].dt.month
prepared_data['day'] = prepared_data['datetime'].dt.day
prepared_data['hour'] = prepared_data['datetime'].dt.hour

prepared_data = prepared_data.drop(columns=['datetime'])

prepared_data['daypart'] = np.vectorize(set_daypart)(prepared_data['hour'])

prepared_data = clean.add_holiday_days(prepared_data)

prepared_data

,intensity,occupation,congestion,longitud,latitud,is_holiday,week,year,month,day,hour,daypart
0,212,2,8,-3.754152,40.401447,1,44,2022,11,1,13,0
1,466,7,36,-3.753719,40.399708,1,44,2022,11,1,13,0
2,180,4,16,-3.745582,40.400560,1,44,2022,11,1,13,0
3,45,0,3,-3.743635,40.400829,1,44,2022,11,1,13,0
4,225,1,8,-3.744638,40.399902,1,44,2022,11,1,13,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4133,480,4,35,NaN,NaN,1,44,2022,11,1,13,0
4134,600,5,37,NaN,NaN,1,44,2022,11,1,13,0
4135,480,3,29,NaN,NaN,1,44,2022,11,1,13,0
4136,1560,15,89,NaN,NaN,1,44,2022,11,1,13,0


In [7]:
from importlib import reload

reload(raw)

xml_data = next(raw.load_dataset('Tráfico. Datos del tráfico en tiempo real', 'xml', 2022, verbose=True))

xml_data

  Found 1 files
    Dataframe size: (4162, 5)


,id,intensity,occupation,congestion,datetime
0,3409,85,0,3,2022-11-01 13:50:04
1,4739,296,6,25,2022-11-01 13:50:04
2,4740,270,4,21,2022-11-01 13:50:04
3,4741,135,1,10,2022-11-01 13:50:04
4,4742,135,8,12,2022-11-01 13:50:04
...,...,...,...,...,...
4157,3817,1260,12,86,2022-11-01 13:50:04
4158,10660,300,2,18,2022-11-01 13:50:04
4159,10662,420,3,26,2022-11-01 13:50:04
4160,10661,1140,11,67,2022-11-01 13:50:04


In [11]:
import lib.app as app

app.prepare_data(xml_data)

,intensity,occupation,congestion,longitud,latitud,is_holiday,week,year,month,day,hour,daypart
0,85,0,3,-3.754152,40.401447,1,44,2022,11,1,13,0
1,296,6,25,-3.753719,40.399708,1,44,2022,11,1,13,0
2,270,4,21,-3.745582,40.400560,1,44,2022,11,1,13,0
3,135,1,10,-3.743635,40.400829,1,44,2022,11,1,13,0
4,135,8,12,-3.744638,40.399902,1,44,2022,11,1,13,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4157,1260,12,86,NaN,NaN,1,44,2022,11,1,13,0
4158,300,2,18,NaN,NaN,1,44,2022,11,1,13,0
4159,420,3,26,NaN,NaN,1,44,2022,11,1,13,0
4160,1140,11,67,NaN,NaN,1,44,2022,11,1,13,0


In [8]:
traffic_stations.to_csv('data/traffic_stations.csv', index=False)